In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pathlib import Path

In [3]:


class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = torch.Tensor(X)
        self.y = torch.LongTensor(y)
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]



In [4]:

class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers, dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout), num_layers=num_layers
        )
        
        self.fc = nn.Linear(input_dim, d_model)
        self.out = nn.Linear(d_model, output_dim)
        
    def forward(self, x):
        x = self.fc(x)
  
        x = self.transformer(x)

        x = self.out(x)

        return x
    


In [8]:
def generate_data(subject_id, task, features, batch_size, seq_len):    
    
    csv_path = './ProcessedDatasets/' + task
    
    csv_files = glob.glob(csv_path + "/*.csv")
    
    train_df_list = []
    test_df_list = []
    
    for file in csv_files:
        if(subject_id in file):
            test_df_list.append(pd.read_csv(file))
#             print(file)
        else:
            train_df_list.append(pd.read_csv(file))
            

    print('Train Subject Trials: ',len(train_df_list))
    print('Test Subject Trials: ',len(test_df_list))
    
    # Concatenate all DataFrames
    train_df   = pd.concat(train_df_list, ignore_index=True)
    test_df   = pd.concat(test_df_list, ignore_index=True)

    
    lb = preprocessing.LabelBinarizer()

    train_labels= train_df.pop('label')
    train_features = train_df

    test_labels= test_df.pop('label')
    test_features = test_df


    all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
    lb.fit(all_class_names)

    train_labels = lb.transform(train_labels)
    test_labels = lb.transform(test_labels)
    
    train_x = train_features.to_numpy()
    train_y = train_labels

    test_x = test_features.to_numpy()
    test_y = test_labels
    
    train_x = train_x[:,:features]
    test_x = test_x[:,:features]
    

    train_dataset = TimeSeriesDataset(train_x, train_y, seq_len)
    test_dataset = TimeSeriesDataset(test_x, test_y, seq_len)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)


    
generate_data("S02","Knot_Tying",10)

Train Subject Trials:  32
Test Subject Trials:  4
(50127, 10)


In [ ]:
# training 

dataset = TimeSeriesDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32)

# best config yet: d_model = 64,  nhead=4, num_layers=2
d_model = 64
nhead=4
num_layers=2

model = TransformerModel(input_dim=X_train.shape[-1], output_dim=7, d_model=d_model, nhead=nhead, num_layers=num_layers)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()




# training loop
for epoch in range(100):
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        x, y = batch
        x = x.to(torch.float32)
        y_pred = model(x)

        # print(y, y_pred)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader):.6f}")


# 2. Create model save path 
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_NAME = str(d_model) + "_" + str(nhead) + "_" + str(num_layers) + "_" + MODEL_NAME
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME



# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")

torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 


print("done saving!")
